# New York EITC

The New York EITC equals 30 percent of the federal EITC, minus the New York household credit.

## Examples

The New York EITC provided up to \$2,000 in 2021, for a joint filer with three children and earnings between about \$25,200 and \$26,100.
It is not smooth because it subtracts the New York household credit, which falls with income in a stepped manner.

In [5]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px


def make_eitc(adults, children):
    sim = IndividualSim(year=2021)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="NY")
    sim.vary("employment_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            ny_eitc=sim.calc("ny_eitc")[0].round(),
            mtr=-sim.deriv(
                "ny_eitc", "employment_income", wrt_target="head"
            ),
            adults=adults,
            children=children,
        )
    )


# Make a table of EITCs for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_eitc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    mtr="Marginal tax rate of NY EITC",
    adults="Adults",
    children="Children",
    ny_eitc="New York EITC",
)

fig = px.line(
    df,
    "employment_income",
    "ny_eitc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="New York Earned Income Tax Credit",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

The NY EITC creates marginal tax rates ranging from -13.5 percent to +6.3 percent, plus spikes corresponding to the NY household credit cliffs.

In [6]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="NY EITC marginal tax rate",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat=".1%")
fig.show()